In [1]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow
from protomech import mess

root_path = Path("..")
tag = "Z_combined_v0"

mech0 = automech.io.read(p_.stereo_mechanism(tag, "json", p_.data(root_path)))

In [2]:
mech0, _ = automech.replace_unstable_products(mech0)

/home/avcopan/proj/project-cyclopentane/code/protomech/src/automech/_mech.py:1279: UserWarning: This function ignores stereochemistry.
  warnings.warn("This function ignores stereochemistry.", stacklevel=1)


In [3]:
automech.resonant_unstable_species_names(mech0)

['S(2258)z',
 'S(2258)e',
 'C5O2nexammzzz',
 'C5O2nexammzze',
 'C5O2nexammzez',
 'C5O2nexammzee',
 'C5O2nexammezz',
 'C5O2nexammeze',
 'C5O2nexammeez',
 'C5O2nexammeee',
 'C5O2jrscln']

In [4]:
# import polars as pl
# from automech.reaction import Reaction

# instab_names = automech.resonant_unstable_species_names(mech0)
# mech0.reactions.filter(
#     # pl.col(Reaction.products).list.set_intersection(instab_names).list.len() > 0
#     pl.col(Reaction.reactants).list.set_intersection(instab_names).list.len() > 0
# )

In [5]:
mess_source_path = p_.mess_source(root_path)
mess_inp_files = list(mess_source_path.glob("*/mess.inp"))
surfs = [mess.surf.from_mess_input(f) for f in mess_inp_files]
surf = mess.surf.combine(surfs)
print(mess_inp_files)
# mess.net.display(surf, mech=mech0)

[PosixPath('../data/mess/source/04_1-2/mess.inp'), PosixPath('../data/mess/source/06_37/mess.inp'), PosixPath('../data/mess/source/05_1-14/mess.inp'), PosixPath('../data/mess/source/03_50-51/mess.inp'), PosixPath('../data/mess/source/06_35/mess.inp'), PosixPath('../data/mess/source/02_05/mess.inp'), PosixPath('../data/mess/source/03_46/mess.inp'), PosixPath('../data/mess/source/05_15/mess.inp'), PosixPath('../data/mess/source/01_01/mess.inp'), PosixPath('../data/mess/source/03_49/mess.inp'), PosixPath('../data/mess/source/07_01/mess.inp'), PosixPath('../data/mess/source/05_16/mess.inp'), PosixPath('../data/mess/source/03_48/mess.inp'), PosixPath('../data/mess/source/05_17/mess.inp'), PosixPath('../data/mess/source/02_1-4/mess.inp'), PosixPath('../data/mess/source/03_1-42/mess.inp'), PosixPath('../data/mess/source/03_54/mess.inp'), PosixPath('../data/mess/source/03_44/mess.inp'), PosixPath('../data/mess/source/06_1-34/mess.inp'), PosixPath('../data/mess/source/08_01/mess.inp'), PosixPat

In [6]:
# Drop species that result in unconnected wells
# (These species are not formed in ring-opening reactions and, without
# stereoisomerization rates, do not affect the model.)
drop_labels = ["C5O2pkpfsdzr0", "C5O2sidwaoze", "C5O2sidwaozz", "C5O2pkpfsdzr1"]
drop_keys = mess.surf.node_keys(surf, labels=drop_labels)
print(drop_keys)
surf = mess.surf.remove_nodes(surf, keys=drop_keys)
mech0 = automech.without_species(mech0, spc_names=drop_labels)

[20, 81, 116, 117]


In [7]:
# Adjust OH addition fake well energy to be 0.1 kcal/mol below the barrier
energy_shift = -0.5  # kcal/mol
label = "FakeW-C5H8(522)+OH(4)"
key = mess.surf.node_key_from_label(surf, label)
node = mess.surf.node_object(surf, key)
node.energy += energy_shift
node.update_mess_body_energy()
print(node.mess_body)

  Species
    RRHO
      Geometry[angstrom]        15
        C         -0.66731       1.04616      -0.04984
        C          0.66791       1.04576      -0.04991
        H         -1.28720       1.93223      -0.11437
        C          1.22790      -0.34580       0.11902
        H          1.28831       1.93147      -0.11454
        C         -0.00035      -1.24165      -0.15224
        H          2.05692      -0.55856      -0.56002
        H          1.60987      -0.47900       1.13814
        C         -1.22811      -0.34514       0.11898
        H         -0.00043      -1.53294      -1.20512
        H         -0.00058      -2.15463       0.44449
        H         -2.05724      -0.55747      -0.56006
        H         -1.61013      -0.47813       1.13809
        O          0.00000       0.00000       4.63218
        H          0.00000       0.00000       3.66188
      Core RigidRotor
        SymmetryFactor          1.0
      End  ! Core
      Frequencies[1/cm]         39
        10

In [8]:
surf = mess.surf.merge_resonant_instabilities(surf, mech0)
mech = automech.merge_resonant_instabilities(mech0, remove=True)
# mess.net.display(surf, mech=mech0)

In [9]:
# Print reagents that are not on the MESS surface
for rgt_names in automech.reaction.reagents(mech.reactions):
    node = next((n for n in surf.nodes if n.names_list == rgt_names), None)
    if node is None:
        print(rgt_names)

In [10]:
surf_dct = mess.surf.split_stoichiometries(surf, mech)
for stoich, stoich_surf in surf_dct.items():
    print(stoich, len(stoich_surf.nodes), len(stoich_surf.edges))

C5H9 4 3
C5H9O2 32 45
C5H9O 18 23
C5H7 7 6
C5H7O2 28 46
C5H11O 4 3
C5H11O2 4 3


In [11]:
mess_calc_path = p_.mess_calc(root_path)
mess_calc_path.mkdir(exist_ok=True)

for stoich, stoich_surf in surf_dct.items():
    print(stoich, len(stoich_surf.nodes), len(stoich_surf.edges))
    stoich_path = mess_calc_path / stoich
    stoich_path.mkdir(exist_ok=True)
    mess_inp_file = stoich_path / "mess.inp"
    mess_inp_file.write_text(mess.surf.mess_input(stoich_surf))

    # Test that it can be read back in
    mess.surf.from_mess_input(mess_inp_file)

C5H9 4 3
C5H9O2 32 45
C5H9O 18 23
C5H7 7 6
C5H7O2 28 46
C5H11O 4 3
C5H11O2 4 3


In [12]:
stoich = "C5H9O"
stoich_surf = surf_dct[stoich]

mess.net.display(stoich_surf, mech=mech, height="1000px")